In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
from scipy.optimize import minimize

In [6]:
attenuation_length = 4.5  # um
absorption = 1 - np.exp(-60e-3 / attenuation_length)
print(absorption * 100)

energy_in = 200   # uJ
energy_absorbed = energy_in * absorption
print("uJ", energy_absorbed)

area = 50 * 50  # um^2



1.324483819280431
uJ 2.648967638560862


In [7]:
hbar = 0.0006582119514  # This is the reduced planck constant in keV/fs
c = 299792458. * 1e-9  # The speed of light in um / fs


def kev_to_wavevec_length(energy):
    return energy / hbar / c


def get_rotmat_around_axis(angleRadian, axis):
    """
    Get a rotation matrix that rotate a vector
    with respect to an rotation_axis by some si111_angle in radian.

    According to the right hand rule,
    if one aligns the thumb with the positive direction of the rotation_axis,
    then a positive si111_angle is direction of your four fingers with
    a hollow fist.

    :param angleRadian:
    :param axis:
    :return:
    """

    # Check the rotation_axis length and normalize it
    if np.linalg.norm(axis) < 1e-6:
        print("The rotation_axis has to be a vector of unit length.")
        return False
    axis /= np.linalg.norm(axis)

    # Step 1: get a vector that is not parallel with the rotation_axis
    new_axis = np.zeros(3, dtype=np.float64)
    new_axis[0] = 1.0

    if (np.linalg.norm(np.cross(new_axis, axis)) <= 1e-8):
        # If this relative is valid, then rotation_axis[0] ~ 1 while  rotation_axis[1] = rotation_axis[2] = 0
        new_axis[0] = 0.0
        new_axis[1] = 1.0

    # print(newAxis)

    # Step 2: remove the projection of the newAxis on the rotation_axis direction
    new_axis -= axis * np.dot(axis, new_axis)
    new_axis /= np.linalg.norm(new_axis)

    # print(newAxis)

    # Step 2: get the other vector though cross project
    new_axis2 = np.cross(axis, new_axis)

    # Construct the matrix
    rotMat = np.zeros((3, 3))
    rotMat += np.outer(axis, axis) + np.cos(angleRadian) * (
            np.outer(new_axis, new_axis) + np.outer(new_axis2, new_axis2))
    rotMat += np.sin(angleRadian) * (np.outer(new_axis2, new_axis) - np.outer(new_axis, new_axis2))

    return rotMat

In [8]:
c * 10

2.9979245800000003

In [23]:
def get_device_rot_mat(angle1, angle2, angle3):
    # angle 1: rotation around the vertical axis
    # angle 2: rotation around the propagatoin axis
    # angle 3: rotation around the horizontal axis

    # Coordiante definition
    #  (horizontal, vertical, propagation)

    rot_axis_1 = np.array([0., 1., 0.])  # Rotation axis of the first motion, i.e., vertical
    rot_axis_2 = np.array([0., 0., 1.])  # Rotation axis of the second motion, i.e.,  propagation
    rot_axis_3 = np.array([1., 0., 0.])  # Rotation axis of the third motion, i.e.,  horizontal

    # Calculate the rotation matrix for each axis
    rot_mat1 = get_rotmat_around_axis(angleRadian=-angle1, axis=rot_axis_1)
    rot_mat2 = get_rotmat_around_axis(angleRadian=-angle2, axis=rot_axis_2)
    rot_mat3 = get_rotmat_around_axis(angleRadian=angle3, axis=rot_axis_3)

    # Rotate the vector from 3 to 2 to 1
    rot_mat_tot = np.dot(rot_mat1, np.dot(rot_mat2, rot_mat3))

    return rot_mat_tot

### Test vectors

In [24]:
horizontal_vec = np.array([1., 0., 0.])
vertical_vec = np.array([0., 1., 0.])
propagation_vec = np.array([0., 0., 1.])

In [25]:
rotation = get_device_rot_mat(angle1=np.deg2rad(0),
                              angle2=np.deg2rad(0),
                              angle3=np.deg2rad(0),
                              )
print(np.dot(rotation, horizontal_vec))

[1. 0. 0.]


### Calculate the Bragg condition

In [26]:
# long_edge : 100
# short_edge : 012
# normal: 0,-1,1
# Notice that:  (1, 0, 3) = (1, 0, 0) + (0, 1, 2) + (0, -1, 1)
#  (0, 0, 3) = (0, 1, 2) + (0, -1, 1)
#  (0, 3, 0) = (0, 1, 2) - 2 * (0, -1, 1)
#  (0, -1 ,3) = ((0, 1, 2) + (0, -1, 1)) - 1/3 * ((0, 1, 2) - 2 * (0, -1, 1))
#             = 2 / 3 * (0, 1, 2) + 5 / 3 * (0, -1, 1)

# Define the parameter for nno
nno100 = np.array([1., 0., 0., ]) * np.pi * 2 / (5.4332 * 1e-4)
nno012 = np.array([0., 0., 1., ]) * np.pi * 2 / (3.1589 * 1e-4)
nno0m11 = np.array([0., 1., 0., ]) * np.pi * 2 / (4.4804 * 1e-4)

nno003 = nno012 + nno0m11
nno030 = nno012 - 2 * nno0m11

nno103 = nno100 + nno012 + nno0m11
nno0m13 = nno003 - 1/3 * nno030
print("nno103", nno103)
print("nno0m13", nno0m13)

# Define the parameter for ngo
ngo100 = np.array([1., 0., 0., ]) * np.pi * 2 / (5.4332 * 1e-4)
ngo012 = np.array([0., 0., 1., ]) * np.pi * 2 / (3.1589 * 1e-4)
ngo0m11 = np.array([0., 1., 0., ]) * np.pi * 2 / (4.4804 * 1e-4)

ngo003 = ngo012 + ngo0m11
ngo030 = ngo012 - 2 * ngo0m11

ngo103 = ngo100 + ngo012 + ngo0m11
ngo0m13 = ngo003 - 1/3 * ngo030
print("ngo103", ngo103)
print("ngo0m13", ngo0m13)

nno103 [11564.4285268  14023.71508611 19890.4216885 ]
nno0m13 [    0.         23372.85847685 13260.28112567]
ngo103 [11564.4285268  14023.71508611 19890.4216885 ]
ngo0m13 [    0.         23372.85847685 13260.28112567]


In [29]:
kin1_len = kev_to_wavevec_length(energy=9.2)
kin2_len = kev_to_wavevec_length(energy=8.3)

kin1 = np.array([0., 0., kin1_len])
kin2 = np.array([0., 0., kin2_len])

### Rotate to find the Bragg peak

In [45]:
# The X-ray is always along the propagation direction, 
# Therefore rotating around the angle 2 is not useful
# Therefore, we want to first rotate angle 1 so that the lattice is inthe
def errorfun(angles):
    error = np.abs(np.linalg.norm(kin1 + np.dot(get_device_rot_mat(angle1=angles[0],
                                                     angle2=np.deg2rad(0),
                                                     angle3=angles[1]), ngo103))
                   - kin1_len)
    return error

res = minimize(fun=errorfun,
               x0=np.array((np.deg2rad(10), np.deg2rad(100))), 
               bounds=[[0, 2 * np.pi],
                       [0, 2 * np.pi]])
print("Minimization success:", res['success'])
print("angle1 (deg)", np.rad2deg(res['x'][0]))
print("angle3 (deg)", np.rad2deg(res['x'][1]))
print("error",errorfun(res['x']))

kout = kin1 + np.dot(get_device_rot_mat(angle1=res['x'][0],
                                        angle2=np.deg2rad(0.),
                                        angle3=res['x'][1],
                                        ), ngo103)
print(kout)

Minimization success: False
angle1 (deg) 8.145572044963691
angle3 (deg) 148.21510863873894
error 1.9556289771571755e-05
[ 12796.7406098  -22397.51616305  38836.97076103]


# Play with the other motor to steer the output wave-vector

In [66]:
# The X-ray is always along the propagation direction, 
# Therefore rotating around the angle 2 is not useful
# Therefore, we want to first rotate angle 1 so that the lattice is inthe
angle2_val = np.deg2rad(-45.5)

def errorfun(angles):
    error = np.abs(np.linalg.norm(kin1 + np.dot(get_device_rot_mat(angle1=angles[0],
                                                     angle2=angle2_val,
                                                     angle3=angles[1]), ngo103))
                   - kin1_len)
    return error

res = minimize(fun=errorfun,
               x0=np.array((np.deg2rad(20), np.deg2rad(100))), 
               bounds=[[0, 2 * np.pi],
                       [0, 2 * np.pi]])
print("Minimization success:", res['success'])
print("angle1 (deg)", np.rad2deg(res['x'][0]))
print("angle2 (deg)", np.rad2deg(angle2_val))
print("angle3 (deg)", np.rad2deg(res['x'][1]))
print("error",errorfun(res['x']))

kout = kin1 + np.dot(get_device_rot_mat(angle1=res['x'][0],
                                        angle2=angle2_val,
                                        angle3=res['x'][1],
                                        ), ngo103)
print("kout", kout)

print("2 theta (deg):", np.rad2deg(np.arccos(np.dot(kin1 / np.linalg.norm(kin1),
                                                    kout / np.linalg.norm(kout)
                                                    ))))

Minimization success: False
angle1 (deg) 35.44995938652372
angle2 (deg) -45.5
angle3 (deg) 186.27435738931945
error 2.073429641313851e-07
kout [2.57954511e+04 1.50606787e+00 3.88369707e+04]
2 theta (deg): 33.5920461932728
